In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import acr
import warnings
import pingouin as pg
import kdephys as kde

from acr.utils import PAPER_FIGURE_ROOT

warnings.filterwarnings('ignore')

plt.style.use('fast')
plt.style.use('/home/kdriessen/gh_master/acr/acr/plot_styles/acr_pub.mplstyle')
#--------------------------------- Import Publication Functions ---------------------------------#
pub_utils = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
from pub_utils import *
data_agg = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')
from data_agg import *

In [ ]:
from acr.utils import acr_animals, sub_swi_exps
for sub in acr_animals:
    exp = sub_swi_exps[sub]
    stim_start, stim_end = acr.stim.stim_bookends(sub, exp)
    stim_dur = (stim_end - stim_start).total_seconds()
    print(f'{sub} {exp} stim_dur: {stim_dur:.2f} seconds')


In [ ]:
sub = 'ACR_34'
exp = acr.utils.sub_swi_exps[sub][0]

# Hypnogram

In [ ]:
h = acr.io.load_hypno_full_exp(sub, exp)
sd_start, stim_start, stim_end, reb_start, full_x_start = acr.info_pipeline.get_sd_exp_landmarks(sub, exp, update=False, return_early=False)

hd = acr.hypnogram_utils.create_acr_hyp_dict(sub, exp)


bl_day = str(full_x_start).split(' ')[0]
fake_x_start = pd.to_datetime(bl_day + ' 09:00:00')
wake_states = ['Wake-Good', 'Transition-to-Wake', 'Sort-Exclude', 'Transition-to-NREM', 'Brief-Arousal', 'Art']
REM_states = ['REM', 'Transition-to-REM']

h.loc[h['state'].isin(wake_states), 'state'] = 'Wake'
h.loc[h['state'].isin(REM_states), 'state'] = 'REM'


# plotting adjustments
h.loc[((h['end_time'] > sd_start) & (h['end_time'] < stim_end)), 'state'] = 'Wake'

reb_end = hd['rebound'].end_time.max()
h = acr.hypnogram_utils.standard_hypno_corrections(h)

# Simple Colored Hypnogram

In [ ]:
h2p = h.trim_select(stim_start-pd.Timedelta('2m'), reb_start+pd.Timedelta('20m'))

In [ ]:
f, ax = kde.plot.main.plot_basic_hypnogram(h2p, style_path='/home/kdriessen/gh_master/acr/acr/plot_styles/acr_pub.mplstyle', single_tone=True)

f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_hypnogram--INDUCTION.png', dpi=600)

In [ ]:
f, ax = kde.plot.main.plot_basic_hypnogram(h2p, style_path='/home/kdriessen/gh_master/acr/acr/plot_styles/acr_pub.mplstyle', single_tone=True)

sb1 = stim_start 
sb2 = stim_start+pd.Timedelta('30m')
# Draw a horizontal scale bar from sb1 to sb2 (1 hour duration)
ax.plot([sb1, sb2], [0.1, 0.1], color='black', linewidth=4)  # Position the scale bar at y=0.1

f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_hypnogram--INDUCTION--ScaleBar.png', dpi=600)

# EMG Plot

In [ ]:
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['xtick.bottom'] = False

In [ ]:
recs = acr.info_pipeline.get_exp_recs(sub, exp)
emg = acr.io.load_concat_raw_data(sub, recs, stores=['EMGr'])
e2p = emg.ts(stim_start-pd.Timedelta('2m'), reb_start+pd.Timedelta('20m')).ch(1).sel(store='EMGr')

In [ ]:
f, ax = plt.subplots(figsize=(20, 0.4))
ax.set_xlim(stim_start-pd.Timedelta('2m'), reb_start+pd.Timedelta('20m'))
ax.set_yticks([])
ax.plot(e2p.datetime, e2p.data, color='#2b3e4b')
ax.set_ylim(-1600, 1800)
print(ax.get_ylim())
f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_emg__INDUCTION.png', dpi=600, bbox_inches='tight', transparent=True, facecolor='none')

# SWA Plot

In [ ]:
chan_to_plot = 13

In [ ]:
_bp = load_raw_bp_df(sub, exp)
bp = make_raw_bp_df_relative_to_baseline(_bp, col_to_use='full_bl', value_to_use='True', state_to_use='NREM', method='mean')

In [ ]:
swa = bp.filter(pl.col('band') == 'delta')
swa_plot = swa.ts(stim_start-pd.Timedelta('2m'), reb_start+pd.Timedelta('20m'))
swa_plot = swa_plot.to_pandas()
swa_plot_o = swa_plot.prb('NNXo').chnl(chan_to_plot)
swa_plot_r = swa_plot.prb('NNXr').chnl(chan_to_plot)

# set the clearly arifactual periods to 0
swa_plot_r.loc[((swa_plot_r['datetime']<reb_start) & (swa_plot_r['bandpower_rel']>0.8)), 'bandpower_rel'] = 0
swa_plot_o.loc[((swa_plot_o['datetime']<stim_start) & (swa_plot_o['bandpower_rel']>0.8)), 'bandpower_rel'] = 0

In [ ]:
# Plot Both Probes - with Yscale!, ch-13
f, ax = plt.subplots(figsize=(24, 8))
ax.set_xlim(stim_start-pd.Timedelta('2m'), reb_start+pd.Timedelta('20m'))
ax.set_yticks([])
ax.plot(swa_plot_o.datetime, swa_plot_o.sm('bandpower_rel', sigma=3)['bandpower_rel'], color=ACR_BLUE, linewidth=4, alpha=1)
ax.plot(swa_plot_r.datetime, swa_plot_r.sm('bandpower_rel', sigma=3)['bandpower_rel'], color=NNXR_GRAY, linewidth=4, alpha=0.85)

ax.set_ylim(0, 3)
#ax.axhline(y=1)
f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_delta--INDUCTION.png', dpi=600, bbox_inches='tight', transparent=True, facecolor='none')

In [ ]:
# Plot Both Probes - with Yscale!
f, ax = plt.subplots(figsize=(24, 8))
ax.set_xlim(stim_start-pd.Timedelta('2m'), reb_start+pd.Timedelta('20m'))
ax.set_yticks([])
ax.plot(swa_plot_o.datetime, swa_plot_o.sm('bandpower_rel', sigma=3)['bandpower_rel'], color=ACR_BLUE, linewidth=4, alpha=1)
ax.plot(swa_plot_r.datetime, swa_plot_r.sm('bandpower_rel', sigma=3)['bandpower_rel'], color=NNXR_GRAY, linewidth=4, alpha=0.85)
ax.set_ylim(0, 3)
ax.axhspan(0, 2, xmin=0.3, xmax=0.32, color='red', alpha=0.7)
ax.plot([stim_start, stim_start+pd.Timedelta('30m')], [2, 2], color='green', linestyle='-', linewidth=3)
f.savefig(f'{PAPER_FIGURE_ROOT}/schems/{sub}_{exp}_delta--INDUCTION_SCALED__30min.png', dpi=600, bbox_inches='tight', transparent=True, facecolor='none')